<a href="https://colab.research.google.com/github/aszapla/Curso-DL/blob/master/3_1_Codigo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sesión 3.1: Redes recurrentes avanzadas

Profesor: [Jorge Calvo Zaragoza](mailto:jcalvo@prhlt.upv.es)

## Ejemplo guiado: traducción neuronal



En esta sesión veremos una arquitectura secuencia a secuencia (sequence-to-sequence, o *seq2seq*) para traducción neuronal de inglés a español. Implementaremos la red neuronal siguiente una arquitectura encoder-decoder. La estructura se ilustra en la siguiente imagen:

![texto alternativo](https://docs.google.com/uc?id=189LP5y8Q2ocge971-nzUUXQb4yFzDdyr)


Como se puede apreciar, vamos a realizar la traducción **a nivel de caracter**.

## Preliminares: generadores

En el caso de tareas relacionadas con redes recurrentes, especialmente cuando se plantean con el enfoque encoder-decoder, es usual tener entradas y salidas de longitud variable. A pesar de que, por definición, las redes recurrentes pueden manejar secuencias de tamaño variable, los paquetes de datos tienen que fijarse a un tipo de dimension concreta: internamente Keras (al igual que otros entornos) trabaja con tensores, los cuales deben tener definidas sus dimensiones. 

Para paliar este problema, existen dos alternativas: crear tantos paquetes como secuencias haya (lo cual es ineficiente en el entrenamiento) o utilizar la técnica de *padding*. El padding consiste en calcular la secuencia de mayor longitud y establecer las dimensiones acorde a este valor. Las secuencias más cortas son rellenadas con valores nulos (típicamente 0).

No obstante, existe una solución intermedia, más elegante, para solventar esta cuestión, a traves de funciones *generadoras*. Las generadoras son funciones que preparan los paquetes de datos. En cada llamada, la generadora prepara el siguiente paquete a tener en cuenta y se *congela*, hasta que es llamado de nuevo para generar otro paquete. Además de la cuestión relacionada con los tensores, las generadoras son de gran utilidad cuando afrontamos una tarea de gran entidad, ya que podría ser inmanejable cargar todo el conjunto de entrenamiento en memoria. De esta forma, sólo se utiliza la memoria que ocupa un único paquete.

En el siguiente código se proporciona un ejemplo intuitivo de este comportamiento (especial atención a las palabras clave **yield** y **next**).

In [0]:

def custom_generator(batch_size):
    X = 'abcdefghij'
    Y = '0123456789'
    
    while True:
      for idx in range(0,len(X),batch_size):
        yield X[idx:idx+batch_size], Y[idx:idx+batch_size]
            
          
generator = custom_generator(batch_size = 4)

for _ in range(5):          
  x,y  = next(generator)
  print(x)
  print(y)
  print('')


abcd
0123

efgh
4567

ij
89

abcd
0123

efgh
4567



## Código Keras paso a paso

Para entrenar un traductor automático neuronal, sólo necesitamos frases en el idioma origen y su correspondiente traducción en el idioma destino. A diferencia de otro tipo de sistemas, no es necesario proporcionar un alineamiento entre los elementos de cada frase. 

Para este ejemplo vamos a utilizar el corpus que se proporciona en [este enlace](http://www.dlsi.ua.es/~jcalvo/data/eng-spa.txt). Este corpus se ha extraído de [este repositorio](http://www.manythings.org/anki/), con diversas opciones para otros pares de idiomas.

In [0]:
import urllib.request # Python3

# Número de pares de secuencias que vamos a considerar
limit_of_sentences = 4000 

# Se descarga el bitexto y se descompone en lineas
bitext_url = "http://www.dlsi.ua.es/~jcalvo/data/eng-spa.txt" 
response = urllib.request.urlopen(bitext_url)
bibtext = response.read().decode('utf-8').splitlines()

# Creamos los corpus de idioma origen y destino
data_input = []
data_output = []

for idx in range( min(len(bibtext), limit_of_sentences) ):
    english, spanish = bibtext[idx].split('\t')
    data_input.append( english )
    data_output.append( spanish )
  
# Comprobación
print(data_input[4])
print(data_output[4])
print()
print(data_input[-4])
print(data_output[-4])

Hi.
Hola.

He made me go.
Él me hizo ir.


Los datos vienen ordenados: la red podría utilizar el orden para *ahorrarse* información en el vector de contexto, lo cual podría ser perjudicial para el aprendizaje. Así pues, barajamos los datos manteniendo el alineamiento entre las frases:

In [0]:
import random

# Enlazamos ambas listas 
bitext_sets = list(zip(data_input, data_output))

# Se barajan y se desenlazan de nuevo
random.shuffle(bitext_sets)
data_input[:], data_output[:] = zip(*bitext_sets)

# Comprobación
print(data_input[4])
print(data_output[4])
print()
print(data_input[-4])
print(data_output[-4])

Has he come?
¿Ha venido?

Grab a seat.
Toma un asiento.


En este punto vamos a incorporar los caracteres de comienzo de secuencia (start of sentence, *sos*) y final de secuencia (end of sentence, *eos*). Vamos a utilizar dos caracteres especiales para indicarlos, con atención a que éstos no pertenezcan a los alfabetos de salida.



In [0]:
# Caracteres de comienzo y final    
output_sos = '<'
output_eos = '>'

# Las datos de salida se transforman para incluir ambos
data_output = [output_sos + output_sentence + output_eos for output_sentence in data_output]

# Comprobación
print(data_output[4])
print()
print(data_output[-4])

<¿Ha venido?>

<Toma un asiento.>


Ahora necesitamos conocer los alfabetos de entrada y salida, especialmente su longitud. Esta información es necesaria para saber la dimension de la representación one hot a utilizar.

In [0]:
# Conjuntos de caracteres
input_alphabet = set()
output_alphabet = set()

for input_sentence in data_input:
    input_alphabet.update(list(input_sentence))

for output_sentence in data_output:
    output_alphabet.update(list(output_sentence))

# Guardamos sus longitudes en variables
input_alphabet_len = len(input_alphabet)
output_alphabet_len = len(output_alphabet)

# Comprobación    
print('Alfabeto de entrada de ' + str(input_alphabet_len) + ' caracteres: ' + str(input_alphabet))    
print('Alfabeto de salida de ' + str(output_alphabet_len) + ' caracteres: ' + str(output_alphabet))    

Alfabeto de entrada de 65 caracteres: {'0', 'b', 'F', 'V', 'w', 'U', 'i', 'B', 'r', '.', 'Y', 'J', '$', 'v', 'A', 'L', 'O', 'l', "'", '7', 'D', 'g', '?', 'p', 'h', 'C', 'n', 'q', 'k', '1', 'e', 'R', '3', 'z', 'y', 'W', 'K', '8', 'T', ':', ',', 'o', 's', 'c', '5', '9', 'M', 'Q', '!', 'j', 'd', 'H', ' ', 'S', 'N', 't', 'E', 'G', 'm', 'f', 'x', 'u', 'I', 'a', 'P'}
Alfabeto de salida de 77 caracteres: {'Á', '0', 'í', 'b', 'F', 'V', 'U', 'ó', 'i', 'B', 'r', '.', 'Y', 'J', 'Ó', 'v', 'Ú', 'L', 'A', 'O', 'l', 'é', '7', 'D', 'g', 'p', '?', 'É', 'h', 'C', '"', 'n', 'q', 'k', '1', 'e', 'R', 'á', '3', '>', 'z', 'y', 'ú', '8', 'T', 'ñ', ',', '¡', '<', ':', 'o', 's', 'c', 'ü', '5', '«', 'M', 'Q', '!', 'j', 'd', 'H', ' ', 'S', 'N', 't', 'E', 'G', 'm', 'f', '»', 'x', 'u', 'I', 'a', '¿', 'P'}


Asignamos un valor numérico identificativo a cada caracter, guardando la conversión entre ambas representaciones para el futuro.

In [0]:
# De caracter a entero
input_alphabet_from_char_to_int = dict([(char, i) for i, char in enumerate(input_alphabet)])
output_alphabet_from_char_to_int = dict([(char, i) for i, char in enumerate(output_alphabet)])    

# De entero a caracter
input_alphabet_from_int_to_char = dict([(i, char) for i, char in enumerate(input_alphabet)])
output_alphabet_from_int_to_char = dict([(i, char) for i, char in enumerate(output_alphabet)])

# Comprobación
print('Identificador 0 para el vocabulario de entrada: ' + str(input_alphabet_from_int_to_char[0]))
print('Identificador de \'a\' para el vocabulario de entrada: ' + str(input_alphabet_from_char_to_int['a']))
print('')
print('Identificador 0 para el vocabulario de salida: ' + str(output_alphabet_from_int_to_char[0]))
print('Identificador de \'a\' para el vocabulario de salida: ' + str(output_alphabet_from_char_to_int['a']))

Identificador 0 para el vocabulario de entrada: 0
Identificador de 'a' para el vocabulario de entrada: 63

Identificador 0 para el vocabulario de salida: Á
Identificador de 'a' para el vocabulario de salida: 74


Construimos los generadores que se usarán para alimentar a la red. Debe tenerse en cuenta que hay que hacer *padding* dentro de cada paquete creado, ya que los tensores a devolver deben tener una dimensión definida.

In [0]:

import numpy as np

# Función auxiliar que convierte listas en paquetes de datos
def batch_to_vectors(data_input, data_output):
  max_input_len = max([len(input_sentence) for input_sentence in data_input])
  max_output_len = max([len(output_sentence) for output_sentence in data_output])

  encoder_input = np.zeros((len(data_input),max_input_len,input_alphabet_len), dtype=np.float)

  for idx_s, input_sentence in enumerate(data_input):        
      for idx_c, char in enumerate(input_sentence):
          encoder_input[idx_s][idx_c][input_alphabet_from_char_to_int[char]] = 1.


  decoder_input = np.zeros((len(data_input),max_output_len,output_alphabet_len), dtype=np.float)
  decoder_output = np.zeros((len(data_input),max_output_len,output_alphabet_len), dtype=np.float)

  for idx_s, output_sentence in enumerate(data_output):        
      for idx_c, char in enumerate(output_sentence):
          decoder_input[idx_s][idx_c][output_alphabet_from_char_to_int[char]] = 1.
          if idx_c > 0:
              decoder_output[idx_s][idx_c-1][output_alphabet_from_char_to_int[char]] = 1.
              
  return encoder_input, decoder_input, decoder_output

# Función generadora
def generator(data_input_full, data_output_full, batch_size=16):  
  while True:
    for idx in range(0,len(data_input_full), batch_size):
      data_input = data_input_full[idx:idx+batch_size]
      data_output = data_output_full[idx:idx+batch_size]
                  
      x,y,t = batch_to_vectors(data_input,data_output)
      yield [x,y], t
    
    
# Inicializamos un generador para la comprobación
check_generator = generator(data_input, data_output)

for _ in range(5):
  [encoder_input, decoder_input], decoder_output = next(check_generator)
  print ('')
  print (encoder_input.shape)
  print (decoder_input.shape)
  print (decoder_output.shape)



(16, 13, 65)
(16, 20, 77)
(16, 20, 77)

(16, 14, 65)
(16, 26, 77)
(16, 26, 77)

(16, 13, 65)
(16, 22, 77)
(16, 22, 77)

(16, 14, 65)
(16, 24, 77)
(16, 24, 77)

(16, 13, 65)
(16, 24, 77)
(16, 24, 77)


Ahora vamos a construir el model neuronal. Cosas a tener en cuenta:
*  ```return_sequences=True``` hace que nos devuelva las predicciones en cada paso.
* ``` return_states=True``` hace que se devuelvan los estados internos.
* Como vamos a utilizar celdas LSTM, los estados internos se componen tanto del estado interno en sí (h) como de la celda memoria (c).

In [0]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

# Encoder
encoder_inputs = Input(shape=(None, input_alphabet_len), name='encoder_input')
encoder_outputs, encoder_h, encoder_c = LSTM(64, return_state=True, name='encoder_output')(encoder_inputs)
context_vector = [encoder_h, encoder_c]

# Decoder
decoder_inputs = Input(shape=(None, output_alphabet_len), name='decoder_input')
decoder_outputs, _, _ = LSTM(64, return_sequences=True, return_state=True, name='decoder_lstm')(decoder_inputs, initial_state=context_vector)
decoder_outputs = Dense(output_alphabet_len, activation='softmax',name='decoder_output')(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')   
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, None, 65)     0                                            
__________________________________________________________________________________________________
decoder_input (InputLayer)      (None, None, 77)     0                                            
__________________________________________________________________________________________________
encoder_output (LSTM)           [(None, 64), (None,  33280       encoder_input[0][0]              
__________________________________________________________________________________________________
decoder_lstm (LSTM)             [(None, None, 64), ( 36352       decoder_input[0][0]              
                                                                 encoder_output[0][1]             
          

## Fase de entrenamiento

Vamos a utilizar 1 % del corpus para validar, dejando el 99 % para entrenar la red.

Para la función de entrenamiento, creamos un generador para el entrenamiento y otro para la validación. No obstante, también obtendremos el paquete completo de validación para monitorizar visualmente qué ocurre.

In [0]:
val_split = 0.01
idx_split = int(len(data_input)*val_split)

data_input_train = data_input[idx_split:]
data_output_train = data_output[idx_split:]

data_input_validation = data_input[:idx_split]
data_output_validation = data_output[:idx_split]

training_generator = generator(data_input_train, data_output_train)
validation_generator = generator(data_input_validation, data_output_validation)

x_val, y_val, t_val = batch_to_vectors(data_input_validation,data_output_validation)

print('Tamaño del conjunto de validación: ' + str(x_val.shape[0]))

Tamaño del conjunto de validación: 40


Entrenamos con los generadores, para lo cual hay que especificar cuántos pasos corresponden a una época: número de datos dividido por el tamaño de los paquetes de datos. Tras cada época, vamos a comprobar qué resultados de traducción proporciona la red.

In [0]:
# 15 épocas 
for epoch in range(15):
    print('Epoca ' + str(epoch))
  
    # Entrenamos durante una época
    model.fit_generator(training_generator, 
                       steps_per_epoch=len(data_input_train)//16, 
                       verbose=2, 
                       epochs=1, 
                       validation_data=[[x_val,y_val],t_val])
    
    # Predecimos sobre el paquete de validación
    batch_prediction = model.predict([x_val,y_val],batch_size=16)
    
    # Comprobamos la traduccion de las 5 primeras secuencias
    for idx,sentence_prediction in enumerate(batch_prediction[:5]):  
        
        # Obtenemos la frase predicha
        raw_predicted_sequence = [output_alphabet_from_int_to_char[char] for char in np.argmax(sentence_prediction,axis=1)]
        
        # Leemos solo hasta el caracter EOS
        predicted_sentence = output_sos         
        for char in raw_predicted_sequence:
            predicted_sentence += char
            if char == output_eos:
                break                    
        
        # Resultado
        print( 'Entrada:\t' + str(data_input_validation[idx]))
        print( 'Predicción:\t' + str(predicted_sentence) )
        print( 'Sal. esperada:\t' + str(data_output_validation[idx]) )
        print('')

Epoca 0
Epoch 1/1
 - 9s - loss: 1.9131 - val_loss: 1.6868
Entrada:	Take mine.
Predicción:	<Eo      eeo.>
Sal. esperada:	<Toma el mío.>

Entrada:	Hi.
Predicción:	<Eo      o>
Sal. esperada:	<Hola.>

Entrada:	Count on it.
Predicción:	<Eoe     e  o...>
Sal. esperada:	<Cuente con eso.>

Entrada:	He's in pain.
Predicción:	<Eo      eee .o.>
Sal. esperada:	<Está sufriendo.>

Entrada:	I am a man.
Predicción:	<Eo      ee o....>
Sal. esperada:	<Soy una persona.>

Epoca 1
Epoch 1/1
 - 8s - loss: 1.6173 - val_loss: 1.4147
Entrada:	Take mine.
Predicción:	<Eo   esoee..>
Sal. esperada:	<Toma el mío.>

Entrada:	Hi.
Predicción:	<Ee   >
Sal. esperada:	<Hola.>

Entrada:	Count on it.
Predicción:	<Eoe  e ee  est.>
Sal. esperada:	<Cuente con eso.>

Entrada:	He's in pain.
Predicción:	<Es e e eeee..o.>
Sal. esperada:	<Está sufriendo.>

Entrada:	I am a man.
Predicción:	<Eo  ee  eera....>
Sal. esperada:	<Soy una persona.>

Epoca 2
Epoch 1/1
 - 8s - loss: 1.4012 - val_loss: 1.2622
Entrada:	Take mine.
Predicción:	

## Fase de predicción

Una vez el modelo está entrenado, podemos usarlo para predecir frases que no se han utilizado en ningún momento. La diferencia con el caso de predecir sobre el conjunto de validación es que no podemos disponer a priori de las entradas del decoder. A cambio, tenemos que hacer una predicción* en linea*, en la cual predecimos simplemente el siguiente símbolo dado el estado actual del decoder y el último símbolo predicho.

Para esto, debemos acceder a las capas internas del modelo creado. Esta funcionalidad se puede realizar en Keras, ya que se puede acceder a las capas de un modelo a partir de su nombre.

In [0]:
for layer in model.layers:
  print (layer.name)

encoder_input
decoder_input
encoder_output
decoder_lstm
decoder_output


Para predecir una frase, necesitamos obtener el context vector. Para ello, necesitamos un modelo que acepte una entrada y nos proporcione el último estado del encoder.

In [0]:
# Creamos el modelo encoder a partir de las capas del modelo entrenado
encoder_model = Model(model.get_layer("encoder_input").input, model.get_layer("encoder_output").output)

 Por otra parte, necesitamos un modelo que acepte un context vector y un elemento, y nos prediga el siguiente elemento de la secuencia a la vez que devuelve el siguiente context vector. 

In [0]:
# Creamos las entradas para el context vector inicial 
decoder_state_input_h = Input(shape=(64,))
decoder_state_input_c = Input(shape=(64,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Obtenemos las referencias a los componentes del decoder
decoder_inputs = model.get_layer("decoder_input").input
decoder_lstm_output, state_h, state_c = model.get_layer("decoder_lstm")(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = model.get_layer("decoder_output")(decoder_lstm_output)

# Creamos un modelo, a partir de las capas del modelo entrenado, que 
# - recibe una secuencia (la última predicción) y un vector de contexto
# - proporciona una predicción y los siguientes estados
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

Con los dos pseudo-modelos creados anteriormente, podemos proceder a predecir una secuencia por completo. Los pasos básicos son:
* Obtener el context vector del encoder
* Predecir los siguientes elementos uno a uno, actualizando el context vector.

Para ello, vamos a definir una función que recibe una frase y realiza la traducción.

In [0]:
# Función que recibe una frase en inglés
def translate_sequence(input_sentence):
  # Creamos el paquete de entrada al decoder, que será una única secuencia de longitud igual al número de caracteres
  input_encoder_seq = np.zeros((1,len(input_sentence),input_alphabet_len), dtype=np.float)
  
  # Convertimos la secuencia a forma codificada one-hot
  for idx_c, char in enumerate(input_sentence):
          input_encoder_seq[0][idx_c][input_alphabet_from_char_to_int[char]] = 1.
  
  # Utilizamos el pseudo-modelo encoder para obtener el context vector
  _, h, c = encoder_model.predict(input_encoder_seq)
  
  # Creamos el context vector concatenando h y c
  states_value = [h,c]

  # Creamos la secuencia de entrada al decoder: sólo necesitamos un elemento
  input_decoder_seq = np.zeros((1, 1, output_alphabet_len))
  # Codificamos el elemento SOS como one-hot
  input_decoder_seq[0, 0, output_alphabet_from_char_to_int[output_sos]] = 1.

  
  # Bucle infinito para crear la secuencia predicha
  decoded_sentence = ''
  while True:
      # Predicción probabilista y siguiente estado
      softmax_prediction, h, c = decoder_model.predict([input_decoder_seq] + states_value)

      # Elegimos el elemento con la mayor probabilidad
      prediction = np.argmax(softmax_prediction[0],axis=1)[0]
      
      # Lo convertimos a caracter
      predicter_character = output_alphabet_from_int_to_char[prediction]

      # Criterio de parada: encontrar el EOS (o tener una secuencia demasiado larga)
      if (predicter_character == output_eos or len(decoded_sentence) > 30):
          break

      # Concatenamos el caracter
      decoded_sentence += predicter_character

      # Actualizamos el context vector para la siguiente iteración       
      states_value = [h, c]
      
      # Actualizamos la entrada del decoder para la siguiente iteración
      input_decoder_seq = np.zeros((1, 1, output_alphabet_len))
      input_decoder_seq[0, 0, output_alphabet_from_char_to_int[predicter_character]] = 1.

  return decoded_sentence

In [0]:
  
print(translate_sequence('This is.'))

Está esto.


La traducción entre lenguajes complejos como el inglés y el español es dificil de modelar. Los resultados obtenidos en este ejemplo están lejos de la fiabilidad que alcanzan los esquemas actuales. No obstante, la formulación de los modelos del estado del arte son similares: la diferencia rádica especialmente en la capacidad de cómputo y la cantidad de datos. No obstante, existen otras cuestiones que hemos obviado en este ejercicio como la búsqueda de hiper-parámetros adecuados, modelos de lenguaje, (word) embeddings o modelos de atención.